# Cuaderno para analisis de correlación de registros de pozo con Python

Este cuaderno de Python tiene como objetivo realizar un análisis de correlación entre las variables relacionadas con un pozo. El objetivo principal es identificar posibles relaciones o dependencias entre estas características del pozo.

El cuaderno utiliza diferentes métodos de correlación, incluyendo el coeficiente de correlación de Pearson, el coeficiente de correlación de Kendall y el coeficiente de correlación de Spearman. Estos métodos nos permiten evaluar la fuerza y la dirección de la relación entre variables numéricas.

Además, se generan mapas de calor (heatmap) que visualizan las correlaciones encontradas. Estos mapas proporcionan una representación gráfica intuitiva de la magnitud de las correlaciones, permitiendo identificar patrones y tendencias entre el RGP y otras características del pozo. El cuaderno es altamente personalizable y permite la selección de las variables específicas a analizar. 

Este cuaderno es una herramienta poderosa para el análisis exploratorio de datos en proyectos relacionados con la industria petrolera, ya que permite comprender mejor las relaciones entre las variables clave de un pozo, facilitando la toma de decisiones informadas y la optimización de los procesos involucrados.

## Bibliotecas a utilizar

Se importan las bibliotecas a utilizar durante el desarrollo de código.

In [2]:
#======================Bibliotecas para graficar=============================#
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.colors
from matplotlib.ticker import StrMethodFormatter
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import seaborn as sns

#======================Bibliotecas para análisis numérico y de datos=========#
import numpy as np
import pandas as pd
import scipy.stats

#======================Bibliotecas para desarrollo de interfaces gráficas====#
import tkinter as tk
from tkinter.filedialog import askopenfilename

## Importar Base de Datos 

En esta sección del script se importan los datos que se van a graficar. Se requiere que el archivo a trabajar este en un formato de texto separado por comas ".csv". Después se genera una tabla resumen con estadigráfos de la base de datos.

In [ ]:
#====================Se crea la ventana con la que se accedará al directorio de la base de datos===#
root = tk.Tk()
root.withdraw()

#====================Dentro de la ventana se selecciona la base de data a abrir====================#
datos = askopenfilename(filetypes = [("csv files", "*.csv")])

#====================Se lee la base de datos y se presenta un resumen con estadígrafos de esta=====#
rgp = pd.read_csv(datos)

## Agrega el nombre de tu pozo o archivo

In [ ]:
pozo = 'Nombre del pozo'

## Nombre de tus variables 

In [ ]:
rgp.columns.tolist()

## Selección de variables
En esta sección puedes escoger las variables a las que quieras analizar su correlación

In [ ]:
rgp_n1 = rgp[[
    'Varaible 1',
    'Varaible 2',
    'Varaible 3',
    'Varaible 4',
    'Varaible 5']]  
rgp_n1.describe().apply(lambda s: s.apply('{0:.4f}'.format)).T


#rgp_n1 = rgp #Para analizar todas las variables

## Estandarización de variables (Estandarización Z-score)
La estandarización es una técnica utilizada para transformar variables numéricas de manera que tengan una media de cero y una desviación estándar de uno. Esta transformación se aplica restando la media de los datos y dividiendo por la desviación estándar. La fórmula para la estandarización z-score es la siguiente:

$variable_{estandarizada} = \frac{{variable - media}}{{desviacion\_estandar}}$

In [ ]:
rgp_n1 = (rgp_n1-rgp_n1.mean())/rgp_n1.std()
rgp_n1.describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

# Matrices de Correlación
Las matrices de correlación son representaciones de las relaciones entre múltiples variables. Se pueden imprimir diagramas de dispersión para visualizar gráficamente estas relaciones. Además, se pueden calcular los coeficientes de correlación, como el coeficiente de correlación de Pearson, Kendall o Spearman, que cuantifican la fuerza y dirección de las relaciones. Interpretar los coeficientes implica comprender que valores altos no implican causalidad. En resumen, las matrices de correlación y los diagramas de dispersión ayudan a entender las relaciones entre variables, mientras que los coeficientes de correlación proporcionan medidas cuantitativas de esas relaciones.

### Funciones

In [1]:
def corr_func(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top right hand corner of a plot.
    """
    r, _ = pearsonr(x, y)
    s, _ = spearmanr(x, y)
    k, _ = kendalltau(x, y)
    fontsize = 30
    ax = ax or plt.gca()
    ax.annotate(f'P = {r:.2f}', xy=(.5, .70), xycoords=ax.transAxes,
                fontsize=fontsize, ha='center')
    ax.annotate(f'S = {s:.2f}', xy=(.5, .50), xycoords=ax.transAxes,
                fontsize=fontsize, ha='center')
    ax.annotate(f'K = {k:.2f}', xy=(.5, .30), xycoords=ax.transAxes,
                fontsize=fontsize, ha='center')

#-----------------------------------------------------------------------------------------------------------------------------
def corr_pairs(df):
    corr_df = df.corr().abs() # Calcula la matriz de correlación y toma los valores absolutos
    corr_pairs = corr_df.unstack().sort_values(ascending=False).drop_duplicates() # Desenrolla la matriz y ordena los valores
    corr_pairs = corr_pairs[corr_pairs.index.get_level_values(0) != corr_pairs.index.get_level_values(1)] # Elimina los pares de variables iguales
    corr_pairs = corr_pairs[(corr_pairs >= 0.7) | (corr_pairs <= -0.7)] # Filtra los valores que sean mayores o iguales a 0.7 o menores o iguales a -0.7
    return corr_pairs

#-----------------------------------------------------------------------------------------------------------------------------
def most_common_vars(df, n):
    """Returns the n most common variables in a DataFrame.
    """
    var_counts = pd.Series(df.columns).value_counts()
    return list(var_counts.head(n).index)

#-----------------------------------------------------------------------------------------------------------------------------
def corrfuncK(x, y, **kwds):
    cmap = kwds['cmap']
    norm = kwds['norm']
    ax = plt.gca()
    ax.tick_params(bottom=False, top=False, left=False, right=False)
    sns.despine(ax=ax, bottom=True, top=True, left=True, right=True)
    r, _ = kendalltau(x, y)
    facecolor = cmap(norm(r))
    ax.set_facecolor(facecolor)
    lightness = (max(facecolor[:3]) + min(facecolor[:3]) ) / 2
    ax.annotate(f"{r:.2f}", xy=(.5, .5), xycoords=ax.transAxes,
                color='white' if lightness < 0.7 else 'black', size=40, ha='center', va='center')
    
#-----------------------------------------------------------------------------------------------------------------------------
def corrfuncS(x, y, **kwds):
    cmap = kwds['cmap']
    norm = kwds['norm']
    ax = plt.gca()
    ax.tick_params(bottom=False, top=False, left=False, right=False)
    sns.despine(ax=ax, bottom=True, top=True, left=True, right=True)
    r, _ = spearmanr(x, y)
    facecolor = cmap(norm(r))
    ax.set_facecolor(facecolor)
    lightness = (max(facecolor[:3]) + min(facecolor[:3]) ) / 2
    ax.annotate(f"{r:.2f}", xy=(.5, .5), xycoords=ax.transAxes,
                color='white' if lightness < 0.7 else 'black', size=40, ha='center', va='center')
    
#-----------------------------------------------------------------------------------------------------------------------------
def corrfuncP(x, y, **kwds):
    cmap = kwds['cmap']
    norm = kwds['norm']
    ax = plt.gca()
    ax.tick_params(bottom=False, top=False, left=False, right=False)
    sns.despine(ax=ax, bottom=True, top=True, left=True, right=True)
    r, _ = pearsonr(x, y)
    facecolor = cmap(norm(r))
    ax.set_facecolor(facecolor)
    lightness = (max(facecolor[:3]) + min(facecolor[:3]) ) / 2
    ax.annotate(f"{r:.2f}", xy=(.5, .5), xycoords=ax.transAxes,
                color='white' if lightness < 0.7 else 'black', size=40, ha='center', va='center')

## Matriz de correlación general

In [ ]:
data = rgp_n1.copy()
g = sns.PairGrid(data)
g.map_upper(corr_func)
g.map_lower(sns.scatterplot)
g.map_diag(sns.histplot)
g.fig.suptitle("Gráfico de correlación de las variables para el pozo "+pozo, fontsize=50, y=1)
for ax in g.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=29)
    ax.set_ylabel(ax.get_ylabel(), fontsize=29)
plt.savefig('Heatmap_correlaciones_'+pozo+'.png', format="png", bbox_inches="tight")
plt.show()

## Variables con correlación alta (positiva y negativa)

In [ ]:
high_corr_pairs = corr_pairs(data)
high_corr_pairs

In [ ]:
most_common_vars(data, 15)# se puede cambiar el número para definir el número de variables con mayor correlación

## Mapa de calor de correlación de Kendall Tau
El coeficiente de correlación de Kendall es una medida no paramétrica utilizada para evaluar la correlación entre dos variables clasificadas o ordenadas. A diferencia del coeficiente de correlación de Pearson, el coeficiente de Kendall no asume una relación lineal y es especialmente útil cuando los datos presentan valores atípicos o cuando la relación entre las variables no es estrictamente monotónica.

La ecuación del coeficiente de correlación de Kendall, denotado como "$\tau$" o "Kendall's tau", se representa

$\tau = \frac{{C - D}}{{\frac{1}{2} n (n - 1)}}$

Donde:

"$C$" es el número de concordancias (pares de elementos que tienen el mismo orden en ambas variables).
"$D$" es el número de discordancias (pares de elementos que tienen un orden diferente en ambas variables).
"$n$" es el tamaño de la muestra (número de observaciones).
El coeficiente de correlación de Kendall varía entre -1 y 1, donde:

Un valor de 1 indica una correlación perfecta positiva (todos los pares de elementos tienen el mismo orden en ambas variables).
Un valor de -1 indica una correlación perfecta negativa (todos los pares de elementos tienen un orden opuesto en ambas variables).
Un valor de 0 indica ausencia de correlación.

In [ ]:
d1 = rgp_n1
g = sns.PairGrid(d1)
g.map_lower(plt.scatter, s=10)
g.map_diag(sns.histplot, kde=False)
g.map_upper(corrfuncK, cmap=plt.get_cmap('seismic'), norm=plt.Normalize(vmin=-1, vmax=1))
g.fig.subplots_adjust(wspace=0.06, hspace=0.06) # equal spacing in both directions
g.fig.suptitle('Mapa de calor de correlación por Kendall y diagramas de dispersión del pozo '+pozo,size=50, y=1.01)
for ax in g.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=30)
    ax.set_ylabel(ax.get_ylabel(), fontsize=30)
plt.savefig('Heatmap_scattergram_Kendall_'+pozo+'.png', format="png", bbox_inches="tight")
plt.show()

## Mapa de calor de correlación de Spearman
El coeficiente de correlación de Spearman es una medida no paramétrica utilizada para evaluar la correlación entre dos variables clasificadas u ordenadas. A diferencia del coeficiente de correlación de Pearson, el coeficiente de Spearman no asume una relación lineal y se basa en los rangos de los datos en lugar de los valores observados.

La fórmula del coeficiente de correlación de Spearman se expresa de la siguiente manera:

$\rho_{spearman} = 1 - \frac{6\sum{d_i^2}}{n(n^2-1)}$

Donde:

"$d_i$" representa las diferencias entre los rangos de los pares de observaciones.
"$n$" es el tamaño de la muestra (número de observaciones).
El coeficiente de correlación de Spearman varía entre -1 y 1, donde:

Un valor de 1 indica una correlación perfectamente monotónica positiva (todos los pares de elementos tienen el mismo orden en ambas variables).
Un valor de -1 indica una correlación perfectamente monotónica negativa (todos los pares de elementos tienen un orden opuesto en ambas variables).
Un valor de 0 indica ausencia de correlación monotónica.

In [ ]:
d1 = rgp_n1
g = sns.PairGrid(d1)
g.map_lower(plt.scatter, s=10)
g.map_diag(sns.histplot, kde=False)
g.map_upper(corrfuncS, cmap=plt.get_cmap('seismic'), norm=plt.Normalize(vmin=-1, vmax=1))
g.fig.subplots_adjust(wspace=0.06, hspace=0.06) # equal spacing in both directions
g.fig.suptitle('Mapa de calor de correlación por Spearman y diagramas de dispersión del pozo '+pozo,size=50, y=1.01)
for ax in g.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=30)
    ax.set_ylabel(ax.get_ylabel(), fontsize=30)
plt.savefig('Heatmap_scattergram_Spearman_'+pozo+'.png', format="png", bbox_inches="tight")
plt.show()

## Mapa de calor de correlación de Pearson
El coeficiente de correlación de Pearson es una medida paramétrica utilizada para evaluar la correlación lineal entre dos variables continuas. Es una medida de la fuerza y la dirección de la relación lineal entre las variables.

La ecuación del coeficiente de correlación de Pearson se representa:

$\rho_{pearson} = \frac{{\sum{(X_i - \bar{X})(Y_i - \bar{Y})}}}{{\sqrt{{\sum{(X_i - \bar{X})^2}} \sum{(Y_i - \bar{Y})^2}}}}$

Donde:

"$X_i$" y "$Y_i$" son los valores de las dos variables.
"$\bar{X}$" y "$\bar{Y}$" son las medias de las dos variables.
El coeficiente de correlación de Pearson varía entre -1 y 1, donde:

Un valor de 1 indica una correlación lineal positiva perfecta (los puntos tienden a estar en una línea ascendente).
Un valor de -1 indica una correlación lineal negativa perfecta (los puntos tienden a estar en una línea descendente).
Un valor de 0 indica ausencia de correlación lineal (no hay una relación lineal aparente entre las variables).

In [ ]:
d1 = rgp_n1
g = sns.PairGrid(d1)
g.map_lower(plt.scatter, s=10)
g.map_diag(sns.histplot, kde=False)
g.map_upper(corrfuncP, cmap=plt.get_cmap('seismic'), norm=plt.Normalize(vmin=-1, vmax=1))
g.fig.subplots_adjust(wspace=0.06, hspace=0.06) # equal spacing in both directions
g.fig.suptitle('Mapa de calor de correlación por Pearson y diagramas de dispersión del pozo '+pozo,size=50, y=1.01)
for ax in g.axes.flat:
    ax.set_xlabel(ax.get_xlabel(), fontsize=30)
    ax.set_ylabel(ax.get_ylabel(), fontsize=30)
plt.savefig('Heatmap_scattergram_Pearson_'+pozo+'.png', format="png", bbox_inches="tight")
plt.show()